### set up the environment and connect to the DB

In [1]:
import sqlite3
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import random

In [2]:
# Initialization
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
MODEL = "gpt-4.1-mini"
openai = OpenAI()

In [3]:
DB = "prices.db"

with sqlite3.connect(DB) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE IF NOT EXISTS prices (city TEXT PRIMARY KEY, price REAL)')
    conn.commit()

### function tools for the LLM

In [8]:
def get_ticket_price(city):
    print(f"DATABASE TOOL CALLED: Getting price for {city}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()
        return f"Ticket price to {city} is ${result[0]}" if result else "No price data available for this city"

In [26]:
def set_ticket_price(city, price):
    print(f"DATABASE TOOL CALLED: set a price for {city}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('INSERT INTO prices (city, price) VALUES (?, ?) ON CONFLICT(city) DO UPDATE SET price = ?', (city.lower(), price, price))
        conn.commit()
    return f"Ticket price is successfully recorded in our database at ${price} for {city}"

In [27]:
def random_price_engine():
    print(f"FUNCTION CALLED: Getting a random price")
    return str(random.randint(0, 10000))

In [28]:
# There's a particular dictionary structure that's required to describe our function:
get_ticket_price_json = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

set_ticket_price_json = {
    "name": "set_ticket_price",
    "description": "set the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
            "price": {
                "type": "integer", 
                "description": "The price of a return ticket to the city"
            }
        },
        "required": ["city", "price"],
        "additionalProperties": False
    }
}

random_price_engine_json = {
    "name": "random_price_engine",
    "description": "randomly generate a ticket price",
    "parameters": {
        "type": "object",
        "properties": {},
        "required": [],
        "additionalProperties": False
    }
}

### chat engine

In [29]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
If there is no price data to the requested city, call the tool "random_price_engine" to get a price 
and then call the tool "set_ticket_price_json" to record the city / price for future reference
"""

In [30]:
tools = [
    {"type": "function", "function": get_ticket_price_json},
    {"type": "function", "function": set_ticket_price_json},
    {"type": "function", "function": random_price_engine_json},
]

In [31]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        print(arguments)
        result = eval(tool_call.function.name + '(**' + str(arguments) + ')')
        responses.append({
            "role": "tool",
            "content": result,
            "tool_call_id": tool_call.id
        })
    return responses

In [32]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    while response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        print(message)
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    return response.choices[0].message.content

### Gradio Interface

> user: what is the price to Bankok? 
- DATABASE TOOL CALLED: Getting price for bankok
- FUNCTION CALLED: Getting a random price
- DATABASE TOOL CALLED: set a price for bankok
> chat: The price of a return ticket to Bankok is $591
> user: what is the price to Bankok? 
- DATABASE TOOL CALLED: set a price for Bankok
> chat: The current price for a return ticket to Bankok is $591

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7878
* To create a public link, set `share=True` in `launch()`.


ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_Q9de0uuc8GKbwi86Qn9Qiwk0', function=Function(arguments='{"city":"bankok"}', name='get_ticket_price'), type='function')])
{'city': 'bankok'}
DATABASE TOOL CALLED: Getting price for bankok
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_dVEk1XLbA9ptbsUS4tmhNysI', function=Function(arguments='{}', name='random_price_engine'), type='function'), ChatCompletionMessageFunctionToolCall(id='call_iSbIWRN3GDR42paJKMOXJ1rO', function=Function(arguments='{}', name='random_price_engine'), type='function')])
{}
FUNCTION CALLED: Getting a random price
{}
FUNCTION CALLED: Getting a random price
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=Non